In [ ]:
#for mongo
from pymongo import MongoClient
import pprint
import pandas as pd

#for cleaning
import re
import string
import pickle

#for modelling
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from sklearn.feature_extraction import text
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

#for recommender
from sklearn.metrics.pairwise import cosine_similarity

# Project 4 Complete Notebook


## 1. Upload Book Data to MongoDB

In [ ]:
#Setup config dictionary to connect to Google Cloud Compute, Mongo

config = {
  'host': '34.94.12.159:27017',
  'username': 'mongo_user',
  'password': 'mongo',
  'authSource': 'book'
}

#connect to MongoClient
client = MongoClient(**config)

#assign database a variable name
db = client.book

#assign collection to database
review_col = db.review
interaction_col = db.interaction
book_col = db.book

In [ ]:
#Format JSON dictionaries into a list
data = [json.loads(line) for line in open('goodreads_books_children.json', 'r')]
data_review = [json.loads(line) for line in open('goodreads_reviews_children.json', 'r')]
data_interaction = [json.loads(line) for line in open('goodreads_interactions_children.json', 'r')]

#Insert lists into mongo
book_col.insert_many(data)
review_col.insert_many(data_review)
interaction_col.insert_many(data_interaction)


## 2. Pull Data from MongoDB with pymongo

### Create List of 500 Most Reviewed 5 Star Illustrated books


#### Create List of Most Positively Reviewed books

In [ ]:
#Get book_id's for top rated, highest volume rated books
pipeline = [
    {'$match': {'rating': 5}},                                     # filter to 5 star books
    {'$group': {'_id': '$book_id', 'num_reviews': {'$sum': 1}}},   # group reviews by book_id and sum 
    {'$project': {'_id': 0, 'book_id': '$_id', 'num_reviews': 1}}, # get book_id and num_reviews
    {'$sort': {'num_reviews': -1}}                                 # sort by num of reviews, descending
]

#put book_ids into a list with highly reviewed books on top
num_reviews_list_desc = list(review_col.aggregate(pipeline))

#put book ids list into a DF
num_review_df = pd.DataFrame(num_reviews_list_desc)


#### Create List of Illustrated books

In [ ]:
#query number of illustrated books
query = { 'description': { "$regex": "illustration*" } }
docs = book_col.count_documents( query )
docs
#12182

#insert illustrated books ids into a DF
illustrated_df = pd.DataFrame(list(book_col.find(query, {"_id":0,"book_id":1, 'title':1})))


#### Create ranked illustrated book list based on positive review volume

In [ ]:
#Inner Merge Illustrated Books DF with Max Num Reviews DF to find most popular illustrated books
illustrated_by_num_review = num_review_df.merge(illustrated_df, how = "inner", on = "book_id")

#save ranked df to csv
illustrated_by_num_review.to_csv("illustrated_df_by_num_review.csv")

#put top 500 book_id's in a list
top_500_book_id = []
for i in range(500):
    top_500_book_id.append(illustrated_by_num_review.iloc[i]["book_id"])


### Pull Review and Book Data of 500 Most Reviewed 5 Star Illustrated books


#### Book Data of 500 Most Reviewed 5 Star Illustrated books

In [ ]:
top_500_book_id = [str(x) for x in top_500_book_id] #change numbers to string type

#aggregate book dfs
book_agg = []

#make dataframe per book
for book in top_500_book_id:
    data = pd.DataFrame(list(book_col.find({'book_id': book}, {"_id":0, "book_id":1, 'description':1, 'title':1, 'publication_year':1, 'image_url':1, 'isbn13':1, 'num_pages':1, 'publisher':1})))
    book_agg.append(data)

#make master dataframe
top_500_book_info_df = pd.concat(book_agg)


#### Review Data of 500 Most Reviewed 5 Star Illustrated books

In [ ]:
#aggregate review dfs
df_agg = []

#make dataframe per book
for book in top_500_book_id:
    data = pd.DataFrame(list(review_col.find({'book_id': book, 'rating':5}, {"_id":0, "book_id":1, 'rating':1, 'review_text':1}).limit(100)))
    data['text'] = data[["book_id",'review_text']].groupby(["book_id"])['review_text'].transform(lambda x: ','.join(x))
    data = data[["book_id",'text']].drop_duplicates()
    df_agg.append(data)

#concat review dfs
top_500_review_df = pd.concat(df_agg)
top_500_review_df

#### Aggregate Book + Review Data

In [ ]:
book_review_df = pd.merge(review_df_str, top_500_book_info_df,  on=['book_id'])
book_review_df.to_csv("book_review_df.csv")


## 3. Clean Data

### Initialize cleaning functions

In [ ]:
# Apply a first round of text cleaning techniques

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    #text = re.sub(r'\b\w{,2}\b', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

### Apply Cleaning to Titles, Review Text, Book Description Text

In [ ]:
#add a cleaned title column to remove duplicates
book_review_df["cleaned_title"] = book_review_df.title
book_review_df["cleaned_title"] = book_review_df.cleaned_title.apply(round1)

#drop duplicate titles
book_review_df_nodup = book_review_df.drop_duplicates(subset="cleaned_title", keep = 'first')
book_review_df_nodup.to_csv("book_review_df_nodup.csv")

#isolate columns we want
#isolate review df, pickle to model with that
review_df = pd.DataFrame(book_review_df[["text"]])

#clean review_df
review_df["text"] = review_df["text"].apply(round1)
review_df["text"] = review_df["text"].apply(round2)
review_df.to_csv("review_df_nodup.csv")

## 4. Analysis + Modelling

### Extract Book Review Impression on Quality of Illustration vs Quality of story

In [ ]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english', ngram_range = (1,2), max_df = .95, min_df = .1)
data_cv = cv.fit_transform(review_df.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = review_df.index
data_dtm

In [ ]:
#See what most common words are
data_dtm_copy = data_dtm
word_df = pd.DataFrame(data_dtm.sum())
word_df = word_df.sort_values(by = 0, ascending = False)
word_df.head(50)

In [ ]:
#isolate words about plot, words about art
data_dtm_copy["words_about_plot"] = data_dtm["story"] + data_dtm["tale"] + data_dtm["tales"] + data_dtm["plot"] + 
                                    data_dtm["stories"] + data_dtm["written"] + data_dtm["writing"] + data_dtm["writer"] + 
                                    data_dtm["text"] +  data_dtm["reader"]

data_dtm_copy["words_about_art"] = data_dtm["illustrations"] + data_dtm["illustration"] + data_dtm["artist"] + 
                                    data_dtm["picture"] + data_dtm["illustrator"] + data_dtm["beautiful"]

total_words_about_art_plot = data_dtm_copy["words_about_plot"] + data_dtm_copy["words_about_art"]

data_dtm_copy["art_over_plot"] = data_dtm_copy["words_about_art"]/total_words_about_art_plot
                                
data_dtm_copy["plot_over_art"] = data_dtm_copy["words_about_plot"]/total_words_about_art_plot

illustration_vs_story_df = data_dtm_copy[["art_over_plot", "plot_over_art"]]
illustration_vs_story_df

In [ ]:
illustration_vs_story_df.to_csv("illustration_vs_story_df.csv")
# Let's pickle it for later use
data_dtm.to_pickle("dtm_review_nodup.pkl")

### Extract Topics from Book Descriptions

#### Helper Functions

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
# Let's create a function to pull out nouns from a string of text
def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

def adj(text):
    '''Given a string of text, tokenize the text and pull out only the  adjectives.'''
    is_adj = lambda pos: pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    adj = [word for (word, pos) in pos_tag(tokenized) if is_adj(pos)] 
    return ' '.join(adj)

def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [ ]:
pickle_in = open("book_clean_df.pkl","rb")
data_clean  = pickle.load(pickle_in)

#### NMF on nouns

In [ ]:
# noun filter on df
data_nouns = pd.DataFrame(data_clean.description.apply(nouns))
data_nouns

# adjective filter on df
data_adj = pd.DataFrame(data_clean.description.apply(adj))
data_adj

# noun adj filter on df
data_nouns_adj = pd.DataFrame(data_clean.description.apply(nouns_adj))
data_nouns_adj

In [ ]:
# Re-add the additional stop words since we are recreating the document-term matrix
stop_noun = ["peter", "pages", "mrs", "beatrix", "potter", "also", "national", "appeal", "everyone", "literature", 
             "nothing", "detailed", "everywhere", "everything", "detailed", "publishers weekly", "adults", 
             "ever", "finally", "parent", "need", "also", "needs", "fans", "asks", "captures", "gift", "five", 
             "detail", "others",  "details", "brought life", "caldecott", "readers", "tale", "tales", "young", 
             "years", "ages", "seuss", "series", "color", "day", "medal", "collier", "review", "seriers", "award", 
             "thing", "stories", "child", "life", "things", "childhood", "year", "world", "award", "winner", "york", 
             "caldecott", "times", "new", "one", "author", "edition", "readers", "reader", "illustrator", "word", 
             "words", "little", "text", "illustration", "illustrations", "story", "picture", "best", "pictures", 
             "children", 'love', 'great', 'book', 'books', 'read', 'reading', 'just', 'like', 'children', 'loved', 
             'time', 'kids', 'fun', 'really', 'reading', 'way', 'favorite', 'page', 'wonderful']

stop_words_noun_agg = text.ENGLISH_STOP_WORDS.union(stop_noun)

# Recreate a document-term matrix with only nouns
tv_noun = TfidfVectorizer(stop_words=stop_words_noun_agg, ngram_range = (1,2), max_df = .6, min_df = .01)
data_tv_noun = tv_noun.fit_transform(data_nouns.description)
data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names())
data_dtm_noun.index = data_nouns.index
data_dtm_noun

In [ ]:
nmf_model = NMF(4)
doc_topic = nmf_model.fit_transform(data_tv_noun)

display_topics(nmf_model, tv_noun.get_feature_names(), 5)

In [ ]:
nmf_model_noun = NMF(20)
doc_topic_noun = nmf_model_noun.fit_transform(data_tv_noun)

display_topics(nmf_model_noun, tv_noun.get_feature_names(), 5)

In [ ]:
H_noun = pd.DataFrame(doc_topic_noun.round(5),
             index = data_clean.index,
             columns = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

H_noun.head(30)

#### Normalize NMF Features

In [ ]:
H_noun["sum"] = H_noun.sum(axis=1)

for num in range(0,20):
    H_noun[num] = H_noun[num]/H_noun["sum"]

H_noun = H_noun.drop(columns = "sum")

In [ ]:
H_noun.columns = ["BOY", "FARM ANIMAL", "CAT", "SCHOOL", "GIRL", 
                        "RABBIT", "POOH", "BABY", "BEAR", "ART", "FAMILY",  
                        "DOG", "DRAGONS", "MICE", "ADVENTURE", "UNKNOWN?", 
                        "BEDTIME", "HISTORY", "ALPHABET", "EDUCATIONAL"]

H_noun.to_csv("h_noun_percentages.csv")

#### NMF on adjectives

In [ ]:
# Re-add the additional stop words since we are recreating the document-term matrix

stop_adj = ["perfect", "best", "new", "good", "young", "old", "little", "beautiful", 'love', 'great', 
            "delightful", 'illustrated', 'read', 'reading', 'just', 'like', 'bear', 'loved', 'tale', 'big', 
            'fun', 'really', 'reading', 'way', 'favorite', 'page', 'wonderful', "book book", "middle", "american"]

stop_words_adj_agg = text.ENGLISH_STOP_WORDS.union(stop_adj)



# Recreate a document-term matrix with only nouns
tv_adj = TfidfVectorizer(stop_words=stop_words_adj_agg, ngram_range = (1,2), max_df = .6, min_df = .02)
data_tv_adj = tv_adj.fit_transform(data_adj.description)
data_dtm_adj = pd.DataFrame(data_tv_adj.toarray(), columns=tv_adj.get_feature_names())
data_dtm_adj.index = data_adj.index
data_dtm_adj


In [ ]:
nmf_model_adj = NMF(6)
doc_topic_adj = nmf_model_adj.fit_transform(data_tv_adj)

display_topics(nmf_model_adj, tv_adj.get_feature_names(), 5)

In [ ]:
H_adj = pd.DataFrame(doc_topic_adj.round(5),
             index = data_clean.index,
             columns = ["CLASSIC", "BLACK AND WHITE", "WHIMSICAL", "SIMPLE", "FUNNY", "DIFFERENT"])
H_adj.head(30)

#### Normalize Features

In [ ]:
H_adj["sum"] = H_adj.sum(axis=1)

H_adj["CLASSIC"] = H_adj["CLASSIC"]/H_adj["sum"]
H_adj["BLACK AND WHITE"] = H_adj["BLACK AND WHITE"]/H_adj["sum"]
H_adj["WHIMSICAL"] = H_adj["WHIMSICAL"]/H_adj["sum"]
H_adj["SIMPLE"] = H_adj["SIMPLE"]/H_adj["sum"]
H_adj["FUNNY"] = H_adj["FUNNY"]/H_adj["sum"]
H_adj["DIFFERENT"] = H_adj["DIFFERENT"]/H_adj["sum"]

H_adj = H_adj.drop(columns = "sum")

H_adj.to_csv("H_adj_percentiles.csv")

In [ ]:
H_noun_adj = pd.merge(H_noun, H_adj, right_index = True, left_index = True)
H_noun_adj.to_csv("H_noun_adj.csv")

#### Concatenate Features DF

In [ ]:
topics_df = pd.read_csv("H_noun_adj_percent.csv")
topics_df = topics_df.fillna(0)

rawdata_df = pd.read_csv("book_review_df_nodup.csv")
rawdata_df = rawdata_df.fillna(0)

illustration_vs_story_df = pd.read_csv("illustration_vs_story_df.csv")
illustration_vs_story_df = illustration_vs_story_df.drop(columns = ["Unnamed: 0"])

topics_df = topics_df.drop(columns = ["title"])

massive_df = pd.concat([topics_df, rawdata_df], axis=1, join='inner')
complete_df = pd.concat([illustration_vs_story_df, massive_df], axis=1, join='inner')

complete_df.to_csv("complete_df.csv")

In [ ]:
similarity_calc_df = pd.DataFrame(complete_df[['art_over_plot', 'plot_over_art', 'EDUCATIONAL', 'BOY', 'GIRL', 
                                               'DIFFERENT', 'CLASSIC', 'num_pages', 'WHIMSICAL', 'SIMPLE', 'FUNNY', 
                                               'ADVENTURE', 'HISTORY', 'ART', 'FAMILY', 'SCHOOL', 'BEDTIME', 
                                               'DRAGONS', 'FARM ANIMAL', 'BEAR', 'POOH', 'DOG', 'CAT', 'MICE', 'RABBIT', 'BABY', 'ALPHABET',  'BLACK AND WHITE', 
                                               'title']].set_index('title'))

similarity_calc_df["BEAR"] = similarity_calc_df["BEAR"] + similarity_calc_df["POOH"]
similarity_calc_df = similarity_calc_df.drop(columns = "POOH")

similarity_calc_df.to_csv("similarity_calc_df.csv")

In [ ]:
mmscaler = MinMaxScaler()
similarity_calc_df["LENGTH_SCALED"] = mmscaler.fit_transform(similarity_calc_df[["LENGTH"]])
similarity_calc_df["LENGTH"] = similarity_calc_df["LENGTH_SCALED"] 
similarity_calc_df = similarity_calc_df.drop(columns= ["LENGTH_SCALED"])

## 5. Recommendation Test Cases : Cosine Similarity

In [ ]:
test_storm = [0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0]
test_mom = [1,0,0,0,1,1,0,200,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0]
test_dad = [1,1,1,1,0,1,0,30,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0]

test_dict = {"test_storm": test_storm, "test_mom": test_mom, "test_dad" : test_dad}
test_dict_df = pd.DataFrame.from_dict(test_dict)
test_dict_df = test_dict_df.transpose()
test_dict_df 

test_dict_df[27] = mmscaler.transform(test_dict_df[[7]])
test_dict_df[7] = test_dict_df[27]
test_dict_df = test_dict_df.drop(columns = [27])
test_dict_df.to_csv("test_df.csv") 

test_dict_df.columns = similarity_calc_df.columns
frames = [test_dict_df, similarity_calc_df]
full_df = pd.concat(frames)
full_df

full_df = full_df.fillna(full_df.mean())

list_of_titles = similarity_calc_df.index.values.tolist()
list_of_titles_full = full_df.index.values.tolist()

In [ ]:
cos_sim_df = pd.DataFrame(cosine_similarity(full_df, full_df).round(5), index=list_of_titles_full, columns=list_of_titles_full)
cos_sim_df.head(3)

In [ ]:
df_storm = pd.DataFrame(cos_sim_df.iloc[[0]])
df_mom = pd.DataFrame(cos_sim_df.iloc[[1]])
df_dad = pd.DataFrame(cos_sim_df.iloc[[2]])

df_storm = df_storm.transpose()
df_mom = df_mom.transpose()
df_dad = df_dad.transpose()

df_storm = df_storm.sort_values("test_storm", ascending = False)
df_mom  = df_mom.sort_values("test_mom", ascending = False)
df_dad = df_dad.sort_values("test_dad", ascending = False)

In [ ]:
df_storm.head(6)

In [ ]:
df_mom.head(6)

In [ ]:
df_dad.head(7)